In [34]:
from imports import *
from encoding_boundaries import GraphEncoder

In [35]:
url = r"D:\Grad\Planify_Dataset\Graph\graphs\boundaries.pkl"

with open(url, 'rb') as f:
    boundaries = pickle.load(f)
    
G = boundaries[1911]
print(G)

Graph with 9 nodes and 9 edges


In [4]:
# Converting networkx boundary graphs to pytorchGeo graphs
Boundaries_pyTorch = []
for b in tqdm(boundaries):
    b_new = from_networkx(b, group_node_attrs=['type', 'centroid'], group_edge_attrs=['distance'])
    
    Boundaries_pyTorch.append(b_new)

Boundaries_pyTorch[0]

100%|██████████| 80787/80787 [00:48<00:00, 1655.33it/s]


Data(edge_index=[2, 26], x=[13, 3], edge_attr=[26, 1])

In [5]:
# Normalization for the centroid coordinates
for b in tqdm(Boundaries_pyTorch, total=len(Boundaries_pyTorch)):
    x = b.x # The feature matrix
    for i in [1, 2]:
        mean = torch.mean(x[:, i])
        std  = torch.std(x[:, i])
        
        normalized_column = (x[:, i] - mean) / std
        b.x[:, i] = normalized_column

100%|██████████| 80787/80787 [00:12<00:00, 6324.21it/s]


In [6]:
import os
checkpoint_dir = "./checkpoints"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    
def save_checkpoint(model, optimizer, epoch):
    checkpoint_path = os.path.join(checkpoint_dir, f'Best_model.pth')
    # Saving model each 15 epochs
#     if epoch % 15 == 0:
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch
    }, checkpoint_path)

In [29]:
edge = int(len(Boundaries_pyTorch) * 0.8)
batch_size = 32

train_dataset = Boundaries_pyTorch[:edge]
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = Boundaries_pyTorch[edge:-10]
val_loader  = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

test_dataset = Boundaries_pyTorch[-10:]
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


print(f"train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")

train: 64629, val: 16148, test: 10


In [36]:
num_of_features = Boundaries_pyTorch[0].x.shape[1]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = GraphEncoder(num_of_features, 64).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.950)
type_criterion = nn.BCEWithLogitsLoss()
centroid_criterion = nn.MSELoss()

In [37]:
def train(model, train_loader, optimizer, type_criterion, centroid_criterion, device):
    model.train()
    total_loss1 = 0.0
    total_loss2 = 0.0
    for data in tqdm(train_loader, total=len(train_loader)):
        features   = data.x.to(device).float()
        edge_index = data.edge_index.to(device)
        
        type_target     = features[:, 0].unsqueeze(1)
        centroid_target = features[:, 1:]
        
        optimizer.zero_grad()
        type_pred, centroid_pred = model(features, edge_index)
        
        # calculate loss
        loss1 = type_criterion(type_pred, type_target)
        loss2 = centroid_criterion(centroid_pred, centroid_target)
        loss  = loss1 + loss2
        
        loss.backward()
        optimizer.step()

        total_loss1 += loss1.item()
        total_loss2 += loss2.item()

    avg_loss1 = total_loss1 / len(train_loader)
    avg_loss2 = total_loss2 / len(train_loader)
    
    return avg_loss1, avg_loss2

def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for data in val_loader:
            features, adj = data.x, data.edge_index
            features = features.to(device).float()
            adj = adj.to(device)

            reconstructed_features = model(features, adj)
            loss = criterion(reconstructed_features, features)

            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    return avg_loss

In [38]:
from copy import deepcopy

learning_rate = 0.0001
num_epochs = 5
patience = 20 # Number of epochs to wait if validation loss doesn't improve
best_val_loss = float('inf')
best_model = None
counter = 0

train_losses = []
val_losses   = []

In [39]:
for epoch in range(num_epochs):
    # Training loop
    train_loss = train(model, train_loader, optimizer, type_criterion, centroid_criterion, device)
    train_losses.append(train_loss)
    
    # # Evaluation loop
    # print('Validating ...')
    # val_loss = validate(model, val_loader, criterion, device)
    # val_losses.append(val_loss)
    
    # Printing and monitoring
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss1: {train_loss[0]:.4f}, Train Loss2: {train_loss[1]:.4f}')
    
    
    # # Early stopping
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    #     best_model = deepcopy(model)
    #     save_checkpoint(best_model, optimizer, epoch)
    #     counter = 0
    # else:
    #     print('Model not saved!')
    #     counter += 1
    #     if counter >= patience:
    #         print(f'Validation loss did not improve for {patience} epochs. Stopping early.')
    #         break
    #     if counter in [3, 5, 7] :
    #         scheduler.step()
    #         print('Learning rate decreased!')

100%|██████████| 2020/2020 [00:23<00:00, 85.24it/s]


Epoch [1/5], Train Loss1: 0.6941, Train Loss2: 0.0472


100%|██████████| 2020/2020 [00:23<00:00, 85.37it/s]


Epoch [2/5], Train Loss1: 0.6932, Train Loss2: 0.0344


100%|██████████| 2020/2020 [00:24<00:00, 83.48it/s]


Epoch [3/5], Train Loss1: 0.6932, Train Loss2: 0.0256


100%|██████████| 2020/2020 [00:24<00:00, 83.60it/s]


Epoch [4/5], Train Loss1: 0.6931, Train Loss2: 0.0225


100%|██████████| 2020/2020 [00:24<00:00, 83.35it/s]

Epoch [5/5], Train Loss1: 0.6931, Train Loss2: 0.0210


In [42]:
features, adj = test_dataset[0].x, test_dataset[0].edge_index    
features = features.to(device).float()
adj = adj.to(device)
type_pred, centroid_pred  = model(features, adj)

In [43]:
type_pred, centroid_pred , print(), features

(tensor([[7.7189e-05],
         [1.3967e-05],
         [1.2893e-07],
         [1.9446e-14],
         [1.0663e-05]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 tensor([[-0.4066, -0.3699],
         [-0.8329,  0.2884],
         [ 0.4597,  0.7337],
         [ 1.0983, -0.6929],
         [-0.7653, -0.0688]], device='cuda:0', grad_fn=<AddmmBackward0>),
 None,
 tensor([[ 0.0000, -0.7212, -0.7753],
         [ 0.0000, -0.7212,  1.0937],
         [ 0.0000,  1.0954,  1.0937],
         [ 0.0000,  1.0954, -0.7753],
         [ 1.0000, -0.7483, -0.6368]], device='cuda:0'))